In [1]:
postingList=[['my', 'dog', 'has', 'flea', \
                          'problems', 'help', 'please'],
                         ['maybe', 'not', 'take', 'him', \
                          'to', 'dog', 'park', 'stupid'],
                         ['my', 'dalmation', 'is', 'so', 'cute', \
                           'I', 'love', 'him'],
                         ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                         ['mr', 'licks', 'ate', 'my', 'steak', 'how',\
                           'to', 'stop', 'him'],
                         ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [3]:
classVec=[0,1,0,1,0,1]

In [9]:
def createVocabList(dataSet):#构造词汇表
    vocabSet=set([])
    for document in dataSet:
        vocabSet=vocabSet.union(set(document))
    return list(vocabSet)

In [10]:
def setOfWords2Vec(vocabList,inputSet):#inputSet中每一个单词是否在词汇表中，在为1，不在为0
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:
            print "the word:%s is not in my Vocabulary!"%word
    return returnVec

In [12]:
myVocabList=createVocabList(postingList)

In [16]:
from numpy import *

In [49]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    p0Num=ones(numWords);p1Num=ones(numWords)#为了便于比较，避免因为某个特性下概率为0 就导致整体概率为0
    p0Denom=2.0;p1Denom=2.0#同上
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]#为1*numWords矩阵，存储属于class 1的每个词汇的出现次数
            p1Denom+=sum(trainMatrix[i])#为一个整数，存储属于class 1的总词汇数
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    p1Vect=log(p1Num/p1Denom)#概率进行乘法过后数值会比较小，取log方便比较（并不需要确切的值，只要不改变相对大小关系），两者相除为每一词汇的出现概率
    p0Vect=log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive
    

In [72]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1=sum(vec2Classify*p1Vec)+log(pClass1)#vec2Classify表示测试集中每个属性（单词表向量）的数量值，p1Vec为利用训练集得出的每个属性的概率值
    p0=sum(vec2Classify*p0Vec)+log(1.0-pClass1)#相乘求和即为测试集中出现单词的概率对数和，再加上class的log，即为概率值
    if p1>p0:
        return 1
    else:
        return 0

In [82]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return returnVec

In [83]:
def textParse(bigString):
    import re
    listOfTokens=re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

In [84]:
docList=[];classList=[];fullText=[]

In [85]:
for i in range(1,26):
    wordList=textParse(open('email/spam/%d.txt' %i).read())
    docList.append(wordList)
    fullText.extend(wordList)
    classList.append(1)
    wordList=textParse(open('email/ham/%d.txt' %i).read())
    docList.append(wordList)
    fullText.extend(wordList)
    classList.append(0)

In [86]:
vocabList=createVocabList(docList)

In [87]:
trainingSet=range(50);testSet=[]

In [88]:
for i in range(10):
    randIndex=int(random.uniform(0,len(trainingSet)))
    testSet.append(trainingSet[randIndex])
    del(trainingSet[randIndex])

In [89]:
trainMat=[];trainClasses=[]

In [90]:
for docIndex in trainingSet:
    trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
    trainClasses.append(classList[docIndex])

In [91]:
p0V,p1V,pSpam=trainNB0(array(trainMat),array(trainClasses))

In [93]:
errorCount=0

In [94]:
for docIndex in testSet:
    wordVector=setOfWords2Vec(vocabList,docList[docIndex])
    if classifyNB(array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
        errorCount+=1

In [95]:
float(errorCount)/len(testSet)

0.1

In [99]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict={}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq=sorted(freqDict.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedFreq[:30]

In [100]:
import feedparser

In [101]:
def localWords(feed1,feed0):
    
    docList=[];classList=[];fullText=[]
    minLen=min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList=textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList=textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList=createVocabList(docList)
    top30Words=calcMostFreq(vocabList,fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    trainingSet=range(2*minLen);testSet=[]
    for i in range(20):
        randIndex=int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat=[];trainClasses=[]
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam=trainNB0(array(trainMat),array(trainClasses))
    errorCount=0
    for docIndex in testSet:
        wordVector=bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount+=1
    print 'the error rate is: ',float(errorCount)/len(testSet)
    return vocabList,p0V,p1V

In [102]:
ny=feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

In [103]:
sf=feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')

In [104]:
vocabList,pSF,PNY=localWords(ny,sf)

the error rate is:  0.2


In [105]:
vocabList,pSF,pNY=localWords(ny,sf)

the error rate is:  0.35


In [109]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[];topSF=[]
    for i in range(len(p0V)):
        if p0V[i]>-6.0:
            topSF.append((vocabList[i],p0V[i]))
        if p1V[i]>-6.0:
            topNY.append((vocabList[i],p1V[i]))
    sortedSF=sorted(topSF,key=lambda pair:pair[1],reverse=True)#根据概率排序
    for item in sortedSF:
        print item[0] #输出词汇
    sortedNY=sorted(topNY,key=lambda pair:pair[1],reverse=True)
    for item in sortedNY:
        print item[0]

In [110]:
getTopWords(ny,sf)

the error rate is:  0.5
women
back
work
six
touch
people
send
guy
abs
meet
alone
put
call
posting
warm
word
making
male
lifting
over
girl
laid
stud
california
girlfriends
times
feel
anyone
white
genuine
bodybuilder
tech
clean
going
stuff
has
pack
sincere
good
partner
older
being
yet
cut
around
know
some
goals
pic
single
welcome
having
serious
sonoma
relieve
month
four
asian
chair
gamut
aug
postings
putting
minded
tickets
entire
school
respectful
pursue
visuals
friendships
seeks
never
hours
great
reposted
conviction
museums
massage
hiking
county
addressed
should
visit
mailme
oils
more
successful
company
downtown
effort
divorced
strain
challenge
movies
nine
awesome
galleries
share
conditi
want
huge
surrounded
how
tension
okay
stress
calves
neck
generou
maybe
anytime
years
disabled
daunting
still
solitary
trying
expectations
actually
willing
image
then
moms
conversations
half
now
accompanying
concerts
weight
fairly
doing
hard
year
attracted
pathetic
attending
imagine
computers
place
invit